In [2]:
# import the necessary packages
from collections import namedtuple
import numpy as np
import json
from shapely.geometry import Polygon
import os

# define the `Detection` object
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])

In [3]:
from shapely.geometry import Polygon
def shape_intersection_over_union(shapeA, shapeB):
    p1 = Polygon(shapeA)
    p2 = Polygon(shapeB)

    # compute the area of intersection rectangle
    interArea = p1.intersection(p2).area

    # compute the area of both the prediction and ground-truth
    boxAArea = p1.area
    boxBArea = p2.area

    # compute the intersection over union by taking the intersection
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [4]:
# load both json folders
def load_array_json_double(folder1, folder2):
    arr1 = []
    path = folder1
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            arr1.append(json.load(open(path+'/'+name)))
    
    arr2 = []
    path = folder2
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            arr2.append(json.load(open(path+'/'+name)))
    return arr1, arr2

In [5]:
from PIL import Image
# load base image folder
def load_array_image(folder):
    arr1 = []
    path = folder
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            #print(os.path.join(root, name))
            #print('practice_dataset_slice-10_Reeven/'+ name)
            arr1.append(Image.open(path+'/'+name))
    return arr1

In [6]:
def process_array_to_tuple(arr):
    output = []
    for i in range(len(arr)):
        for j in range(len(arr[i]['shapes'])):
            temp = []
            if i >= len(output):
                output.append(temp)
            arrTuple = array_to_tuple(arr[i]['shapes'][j]['points'])
            output[i].append(arrTuple)
    return output
    
def array_to_tuple(arr):
    output = []
    for i in range(len(arr)):
        x = arr[i][0]
        y = arr[i][1]
        output.append((x,y))
    return output

In [7]:
# load data
arr1, arr2 = load_array_json_double('json set 1', 'json set 2')
base_images = load_array_image('bg images')

In [67]:
from PIL import Image, ImageEnhance, ImageFont, ImageDraw, ImageColor
import statistics

def display_label_data(arr1, arr2, base_images, img_num, showImage):
    # made by Reeven Ajero
    arr1Tuple = process_array_to_tuple(arr1)
    arr2Tuple = process_array_to_tuple(arr2)   
        
    arr1_dic = get_label_average_pos(arr1, arr1Tuple, img_num)
    arr2_dic = get_label_average_pos(arr2, arr2Tuple, img_num)
    used_j = []
    iou = []
    iou_dic = []
    label_no_pair = 0

    # this block chooses one label from the first array and scanns the entier second array for a match.
    # uses average XY position and labels to decide
    for i in range(len(arr1_dic)):
        matching_j = -1
        for j in range(len(arr2_dic)):
            ratio_x = arr1_dic[i][1][0]/arr2_dic[j][1][0]
            ratio_y = arr1_dic[i][1][1]/arr2_dic[j][1][1]
            ratio = (ratio_x + ratio_y) / 2
            #print(f'{i}, j:{j}, ratio:{ratio}, not in used j:{j not in used_j}, label matching:{arr1_dic[i][0] == arr2_dic[j][0]}')
            # if statement that decides the matching label
            if ratio >= 0.9 and ratio <= 1.1 and j not in used_j:
                if arr1_dic[i][0] == arr2_dic[j][0] or (arr1_dic[i][0] == 'neural_rosette' and arr2_dic[j][0] == 'bad_rosette') or (arr1_dic[i][0] == 'bad_rosette' and arr2_dic[j][0] == 'neural_rosette'):
                    #print(f'chosen:{j}\n')
                    matching_j = j
                    used_j.append(j)
                    iou_value = shape_intersection_over_union(arr1[img_num]['shapes'][i]['points'], arr2[img_num]['shapes'][j]['points'])
                    iou.append(iou_value)
                    iou_dic.append((arr1_dic[i][0],iou_value))
                    break
        
        if matching_j == -1:
            #print(f'chosen:{None}\n')
            label_no_pair += 1            
            iou.append(None)

    # Draw section
    clear = base_images[img_num].convert('RGB').copy()
    draw = ImageDraw.Draw(clear)
    font = ImageFont.truetype("arial.ttf", 50)  
    pos_used = []
    for j in range(len(arr2[img_num]['shapes'])):
        draw.polygon(xy=arr2Tuple[img_num][j], fill=None, outline=(0,0,255), width=5)
    for i in range(len(arr1[img_num]['shapes'])):
        draw.polygon(xy=arr1Tuple[img_num][i], fill=None, outline=(255,0,0), width=5)
        # Text drawing and positioning
        # checks if the text placed previously is too close to the new one
        # new text will be pushed upward
        if(iou[i] != 0 and iou[i] != None):
            pos = arr1_dic[i][1]
            offset = 100
            push_dist = 150
            pos=(pos[0]-250,pos[1])
            for k in range(len(pos_used)):
                if pos[0] <= pos_used[k][0]+offset and pos[0] >= pos_used[k][0]-offset and pos[1] >= pos_used[k][1]-offset and pos[1] <= pos_used[k][1]+offset:
                    pos=(pos[0],pos[1]-push_dist)   
            pos_used.append(pos)     
            draw.text(xy=pos, text= arr1_dic[i][0] + ' IOU: ' + str(iou[i])[0:4], font=font, fill=(0, 255, 0))

    if showImage:
        clear.show()
    return iou_dic, label_no_pair

# works out the average position of a label and places it in a dictionary that contains the label and the xy tuple
def get_label_average_pos(arr1, arr1Tuple, img_num):
    arr1_dic = []
    for i in range(len(arr1Tuple[img_num])):
        x=[]
        y=[]
        for j in range(len(arr1Tuple[img_num][i])):        
            x.append(arr1Tuple[img_num][i][j][0])     
            y.append(arr1Tuple[img_num][i][j][1])
        mean_x = np.mean(x)
        mean_y = np.mean(y)
        arr1_dic.append((arr1[img_num]['shapes'][i]['label'],(mean_x,mean_y)))
  
    return arr1_dic

In [68]:
iou_set = []
iou_no_pair = 0
for i in range(len(arr1)):
    iou, no_pair = display_label_data(arr1, arr2, base_images, i, False)
    iou_set.append(iou)
    iou_no_pair += no_pair

In [69]:
rosettes_iou = []
lumen_iou = []
stain_iou = []
for i in range(len(iou_set)):
    for j in range(len(iou_set[i])):
        if iou_set[i][j][0] == 'neural_rosette' or iou_set[i][j][0] == 'bad_rosette':
            rosettes_iou.append(iou_set[i][j][1])
        elif iou_set[i][j][0] == 'lumen':
            lumen_iou.append(iou_set[i][j][1])
        else:
            stain_iou.append(iou_set[i][j][1])

In [70]:
rosettes_iou_mean = np.mean(rosettes_iou)
lumen_iou_mean = np.mean(lumen_iou)
stain_iou_mean = np.mean(stain_iou)
rosettes_iou_std = np.std(rosettes_iou)
lumen_iou_mean_std = np.std(lumen_iou)
stain_iou_mean_std = np.std(stain_iou)

In [71]:
print(rosettes_iou_mean)
print(lumen_iou_mean)
print(stain_iou_mean)

0.9706647876873509
0.7835591358893509
0.9654339241146184


In [72]:
print(rosettes_iou_std)
print(lumen_iou_mean_std)
print(stain_iou_mean_std)

0.11461517060114144
0.30738530918400697
0.021841017321797135


In [73]:
print(iou_no_pair)

50
